In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import pickle

In [2]:
model = load_model('model.h5')

In [3]:
with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

with open('label_encoder.pkl','rb') as file:
    label_encoder = pickle.load(file)

with open('one_hot_encoder.pkl','rb') as file:
    one_hot_encoder = pickle.load(file)

In [4]:
input_data = {
    'CreditScore': [600],
    'Geography': ['France'],
    'Gender': ['Male'],
    'Age': [40],
    'Tenure': [3],
    'Balance': [60000],
    'NumOfProducts': [2],
    'HasCrCard': [1],
    'IsActiveMember': [1],
    'EstimatedSalary': [50000]
}

In [5]:
data = pd.DataFrame(input_data)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [6]:
numerical_columns = [features for features in data.columns if data[features].dtype!='object']
numerical_columns

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary']

In [7]:
data[numerical_columns] = scaler.transform(data[numerical_columns])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,-0.501365,France,Male,-0.108801,-0.685439,-0.366556,0.713092,0.649021,1.094722,-0.877256


In [8]:
data['Gender'] = label_encoder.transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,-0.501365,France,1,-0.108801,-0.685439,-0.366556,0.713092,0.649021,1.094722,-0.877256


In [9]:
one_hot_encoded_array = one_hot_encoder.transform([[input_data['Geography'][0]]]).toarray()
one_hot_encoded_array

c:\Users\azhar\OneDrive\Desktop\ANNProject\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array([[1., 0., 0.]])

In [10]:
one_hot_encoded_dataframe = pd.DataFrame(one_hot_encoded_array,columns=one_hot_encoder.get_feature_names_out(['Geography'])).astype(int)
one_hot_encoded_dataframe

,Geography_France,Geography_Germany,Geography_Spain
0,1,0,0


In [11]:
data_transformed = pd.concat([data.drop(columns='Geography'),one_hot_encoded_dataframe],axis=1)
data_transformed

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,-0.501365,1,-0.108801,-0.685439,-0.366556,0.713092,0.649021,1.094722,-0.877256,1,0,0


In [12]:
prediction = model.predict(data_transformed)
prediction

1/1 [==============================] - 0s 301ms/step


array([[0.20655614]], dtype=float32)

In [13]:
prediction_proba=prediction[0][0]

In [14]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.


In [20]:
one_hot_encoder.categories_[0]

array(['France', 'Germany', 'Spain'], dtype=object)

In [18]:
label_encoder.classes_

array(['Female', 'Male'], dtype=object)